# Retail Sales Analysis — EDA

**Date:** 2025-09-24  
**Author:** *Luis Renteria Lezano*
## Executive Summary
- **Goal:** Explore sales patterns to identify top-performing product categories and stores.
- **Source:** This analysis uses the Online Retail dataset from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/online+retail)
- **Data:** `data/raw/Online Retail.xlsx`.


>This notebook focuses only on data manipulation and descriptive analysis with **Pandas**.  
>If you want to see data **visualization** (plots and charts), please check the folder: <br>
>`03-data-visualization/pandas-retail-sales-analysis` <br>
>There you’ll find complementary notebooks with plots built using **Matplotlib** and **Seaborn**.

## 0. Reproducibility & Setup
- Pin versions in `requirements.txt`.
- Keep raw data immutable; write cleaned/derived data to `data/processed/`.
- Export final tables to `reports/tables/`.

## 1. Introduction
This dataset contains transactional data from an online retail store.  
Each row represents a line item of a customer’s invoice, including product, quantity, price, and customer details.

**Columns available:**
- `InvoiceNo` → Transaction unique identifier, if starts with 'C' indicates cancellations.
- `StockCode` → Product identifier.
- `Description` → Product name.
- `Quantity` → Number of units purchased (can be negative for returns).
- `InvoiceDate` → Date and time of the transaction.
- `UnitPrice` → Price per unit of the product.
- `CustomerID` → Unique identifier for the customer.
- `Country` → Country where the transaction took place.

**Purpose of this EDA:**
- Understand the structure and quality of the dataset.
- Explore sales trends, products, customers, and country-level behavior.
- Detect anomalies (e.g., negative quantities, missing customers, cancelled invoices).
- Generate insights that could support business decisions (e.g., top-selling products, most valuable customers, countries with highest revenue).

In [448]:
import pandas as pd
from pathlib import Path

raw_path = Path('../data/raw/Online Retail.xlsx')
df_raw = pd.read_excel(raw_path)

processed_path = Path('../data/processed/online_retail.csv')
df_raw.to_csv(processed_path, index=False)

parse_dates = ["InvoiceDate"]
dtype_map = {
    "InvoiceNo": "string",
    "StockCode": "string",
    "Description": "string",
    "Quantity": "Int64",     # allow NA
    "UnitPrice": "float64",
    "CustomerID": "Int64",
    "Country": "string",
}

df = pd.read_csv(processed_path, dtype=dtype_map, parse_dates=parse_dates)
df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom


## 2. Dataset Overview
This section help us gain a better insight into the data, including the data type of each column, the shape of the dataset (541909, 8), and the number of unique values per column. It is important to note that `InvoiceDate` was already converted to datetime in the setup. 

In [449]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  string        
 1   StockCode    541909 non-null  string        
 2   Description  540455 non-null  string        
 3   Quantity     541909 non-null  Int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  Int64         
 7   Country      541909 non-null  string        
dtypes: Int64(2), datetime64[ns](1), float64(1), string(4)
memory usage: 34.1 MB


In [450]:
df.nunique()

InvoiceNo      25900
StockCode       4070
Description     4223
Quantity         722
InvoiceDate    23260
UnitPrice       1630
CustomerID      4372
Country           38
dtype: int64

## 3. Data Cleaning
Tasks to perform:
- Check null & duplicates values in each column
- Normalize data types (already done in loader)
- Basic sanity filters (e.g., negative quantities & prices, cancellation invoices starting with 'C')

In [451]:
# Nulls
nulls_count = df.isna().sum().sort_values(ascending=False).sum()
print(f'Null count: {nulls_count} from columns {df.isna().any()[df.isna().any() == True].index.to_list()}')

# Duplicates
dup_values = df.duplicated().sum()
print(f'Duplicated records: {dup_values}')

# Quantity & Price negative values check
neg_qty = df[df['Quantity'] <= 0].shape[0]
neg_price = df[df['UnitPrice'] <= 0].shape[0]
print(f'Negaive quantity count: {neg_qty}')
print(f'Negative prices count: {neg_price}')

# Cancelled Orders
can_orders = df["InvoiceNo"].str.startswith("C").sum()
print(f'Cancelled Orders: {can_orders}')

Null count: 136534 from columns ['Description', 'CustomerID']
Duplicated records: 5268
Negaive quantity count: 10624
Negative prices count: 2517
Cancelled Orders: 9288


Since it is not possible to apply imputation techniques to `CustomerID` or `Description`, these rows will be dropped. The same applies to duplicate rows and records with negative quantities or prices, which are considered inconsistent. Moreover, a new column called `Revenue` which will be the product of `Quantity` and `UnitPrice` will be added, and `Date` column will be extracted from `InvoiceDate`.

In [452]:
df_1 = df.dropna().drop_duplicates()
clean_df = df_1[(df_1['Quantity'] > 0) & (df_1['UnitPrice'] > 0)].copy()
clean_df['Revenue'] = clean_df['UnitPrice'] * clean_df['Quantity']
clean_df['Date'] = pd.to_datetime(clean_df['InvoiceDate'].dt.date)
clean_df.head(3)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue,Date
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.3,2010-12-01
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,2010-12-01
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.0,2010-12-01


> A cleaned version of the dataset will be stored in `data/processed` to be used in the future or other notebooks. <br>
> For clarity and simplicity in the subsequent analysis, the cleaned dataset `clean_df` is reassigned to `df`. This avoids clutter in the code and maintains consistency with common conventions.

In [453]:
cleaned_path = Path('../data/processed/online_retail_clean.csv')
clean_df.to_csv(cleaned_path, index=False)
df = clean_df

## 4. Descriptive Statistics

In [454]:
df[['Quantity','UnitPrice', 'Revenue']].describe().round(2)

,Quantity,UnitPrice,Revenue
count,392692.0,392692.00,392692.0
mean,13.12,3.13,22.63
std,180.49,22.24,311.1
min,1.0,0.00,0.0
25%,2.0,1.25,4.95
50%,6.0,1.95,12.45
75%,12.0,3.75,19.8
max,80995.0,8142.75,168469.6


In [455]:
# Date range
min_date = df['InvoiceDate'].min()
max_date = df['InvoiceDate'].max()

# Entities
n_tx = df["InvoiceNo"].nunique()
n_customers = df["CustomerID"].nunique()
n_products = df["StockCode"].nunique()
n_countries = df["Country"].nunique()

print(f"Date range: {min_date} -> {max_date}")
print(f"Unique invoices: {n_tx}, customers: {n_customers}, products: {n_products}, countries: {n_countries}")

Date range: 2010-12-01 08:26:00 -> 2011-12-09 12:50:00
Unique invoices: 18532, customers: 4338, products: 3665, countries: 37


## 5. Product Analysis
The following analysis for *products* will be performed in this section using Pandas only.
- Top products by quantity sold
- Top products by revenue
- Most frequent products (by line items count): Items sold frequently but not necessarily in big quantities

In [456]:
top_products_quantity = df.pivot_table(values='Quantity', index=['StockCode', 'Description'], aggfunc='sum').sort_values('Quantity', ascending=False).iloc[:10]
top_products_quantity.to_csv('../reports/tables/top_products_quantity.csv')
top_products_quantity

,,Quantity
StockCode,Description,
23843,"PAPER CRAFT , LITTLE BIRDIE",80995
23166,MEDIUM CERAMIC TOP STORAGE JAR,77916
84077,WORLD WAR 2 GLIDERS ASSTD DESIGNS,54319
85099B,JUMBO BAG RED RETROSPOT,46078
85123A,WHITE HANGING HEART T-LIGHT HOLDER,36706
84879,ASSORTED COLOUR BIRD ORNAMENT,35263
21212,PACK OF 72 RETROSPOT CAKE CASES,33670
22197,POPCORN HOLDER,30919
23084,RABBIT NIGHT LIGHT,27153


In [457]:
df_nopost_nomanual = df[(df['Description'] != 'Manual') & (df['Description'] != 'POSTAGE')]
top_products_revenue = df_nopost_nomanual.pivot_table(values='Revenue', index=['StockCode', 'Description'], aggfunc='sum').sort_values('Revenue', ascending=False).iloc[:10]
top_products_revenue.to_csv('../reports/tables/top_products_revenue.csv')
top_products_revenue

,,Revenue
StockCode,Description,
23843,"PAPER CRAFT , LITTLE BIRDIE",168469.6
22423,REGENCY CAKESTAND 3 TIER,142264.75
85123A,WHITE HANGING HEART T-LIGHT HOLDER,100392.1
85099B,JUMBO BAG RED RETROSPOT,85040.54
23166,MEDIUM CERAMIC TOP STORAGE JAR,81416.73
47566,PARTY BUNTING,68785.23
84879,ASSORTED COLOUR BIRD ORNAMENT,56413.03
23084,RABBIT NIGHT LIGHT,51251.24
79321,CHILLI LIGHTS,46265.11


In [458]:
df_nopost_nomanual = df[(df['Description'] != 'Manual') & (df['Description'] != 'POSTAGE')]
most_frequent_prod = df_nopost_nomanual[['StockCode', 'Description']].value_counts().reset_index().set_index(['StockCode', 'Description']).iloc[:10]
most_frequent_prod.to_csv('../reports/tables/most_frequent_prod.csv')
most_frequent_prod

,,count
StockCode,Description,
85123A,WHITE HANGING HEART T-LIGHT HOLDER,2016
22423,REGENCY CAKESTAND 3 TIER,1713
85099B,JUMBO BAG RED RETROSPOT,1615
84879,ASSORTED COLOUR BIRD ORNAMENT,1395
47566,PARTY BUNTING,1389
20725,LUNCH BAG RED RETROSPOT,1303
22720,SET OF 3 CAKE TINS PANTRY DESIGN,1152
20727,LUNCH BAG BLACK SKULL.,1078
21212,PACK OF 72 RETROSPOT CAKE CASES,1050


## 6. Customer Analysis (RFM)
The following analysis for *customers* will be performed in this section using Pandas only.
- Recency: how much time has been since their last purcharse?
- Frequency: how often do they make purcharses?
- Monetary: how much did they spend total?

In [459]:
# RECENCY:
# Considering this is being analyzed on 2011-12-09, as this was the last date recorded:
mod_df = df.loc[:]
mod_df['recency'] = pd.to_datetime('2011-12-09') - mod_df['Date']
mod_df['Last date of purcharse'] = mod_df['Date']
rfm_recency_df = mod_df.groupby('CustomerID')[['Last date of purcharse', 'recency']].agg({'Last date of purcharse': 'max', 'recency': 'min'}).sort_values('Last date of purcharse', ascending=False)
rfm_recency_df.to_csv('../reports/tables/rfm_recency_df.csv')
rfm_recency_df

,Last date of purcharse,recency
CustomerID,,
14051,2011-12-09,0 days
18102,2011-12-09,0 days
12662,2011-12-09,0 days
14441,2011-12-09,0 days
17754,2011-12-09,0 days
...,...,...
14237,2010-12-01,373 days
16583,2010-12-01,373 days
17968,2010-12-01,373 days


In [460]:
# FREQUENCY:
# Considering the whole dataset from 2010-12-01 08:26:00 to 2011-12-09 12:50:00
mod_df = df.loc[:]
rfm_frequency_df = mod_df.groupby(['CustomerID'])[['InvoiceNo']].nunique().sort_values('InvoiceNo', ascending=False)
rfm_frequency_df.to_csv('../reports/tables/rfm_frequency_df.csv')
rfm_frequency_df.head(5)

,InvoiceNo
CustomerID,
12748,209
14911,201
17841,124
13089,97
14606,93


In [461]:
# MONETARY:
# Considering the whole dataset from 2010-12-01 08:26:00 to 2011-12-09 12:50:00
mod_df = df.loc[:]
rfm_monetary_df = mod_df.groupby('CustomerID')[['Revenue']].sum().sort_values('Revenue', ascending=False)
rfm_monetary_df.to_csv('../reports/tables/rfm_monetary_df.csv')
rfm_monetary_df.head(5)

,Revenue
CustomerID,
14646,280206.02
18102,259657.3
17450,194390.79
16446,168472.5
14911,143711.17


## 7. Country Analysis
The following analysis for *countries* will be performed in this section using Pandas only.
- Review and clean inconsistent or non-country values in the `Country` column.
- Transactions by country
- Revenue by country

In [409]:
display(df['Country'].values.unique())
country_analysis_df = df[(df['Country'] != 'Unspecified') & (df['Country'] != 'European Community')]

<StringArray>
[      'United Kingdom',               'France',            'Australia',
          'Netherlands',              'Germany',               'Norway',
                 'EIRE',          'Switzerland',                'Spain',
               'Poland',             'Portugal',                'Italy',
              'Belgium',            'Lithuania',                'Japan',
              'Iceland',      'Channel Islands',              'Denmark',
               'Cyprus',               'Sweden',              'Finland',
              'Austria',               'Greece',            'Singapore',
              'Lebanon', 'United Arab Emirates',               'Israel',
         'Saudi Arabia',       'Czech Republic',               'Canada',
          'Unspecified',               'Brazil',                  'USA',
   'European Community',              'Bahrain',                'Malta',
                  'RSA']
Length: 37, dtype: string

In [462]:
txns_per_country = country_analysis_df.groupby('Country')[['InvoiceNo']].nunique().sort_values('InvoiceNo', ascending=False)
txns_per_country.to_csv('../reports/tables/txns_per_country.csv')
txns_per_country.head(5)

,InvoiceNo
Country,
United Kingdom,16646
Germany,457
France,389
EIRE,260
Belgium,98


In [463]:
rev_per_country = country_analysis_df.groupby('Country')[['Revenue']].sum().sort_values('Revenue', ascending=False)
rev_per_country.to_csv('../reports/tables/rev_per_country.csv')
rev_per_country.head(5)

,Revenue
Country,
United Kingdom,7285024.644
Netherlands,285446.34
EIRE,265262.46
Germany,228678.4
France,208934.31


## 8. Additional Insights
- Top 3 Customers per day in the last week (by Revenue)
- Monthly Revenue
- Pareto Principle for customers

In [464]:
last_week = df[df['Date'] >= '2011-12-02']
top_3_cx_last_week = last_week.pivot_table(values='Revenue', index=['Date', 'CustomerID'], aggfunc='sum').sort_values(['Date', 'Revenue'], ascending=[False, False]).groupby(level='Date').head(3)
top_3_cx_last_week.to_csv('../reports/tables/top_3_cx_last_week.csv')
top_3_cx_last_week

Revenue
Date       CustomerID          
2011-12-09 16446       168469.6
           12433        2638.69
           14051         1203.9
2011-12-08 18102        11016.1
           16210         3599.4
           17949         1944.0
2011-12-07 16000        12393.7
           14646       11477.42
           17511        4949.24
2011-12-06 17389        5134.88
           15061        3895.86
           13694        3131.16
2011-12-05 14096        8935.98
           16684        5401.98
           13089        3705.48
2011-12-04 17131        1067.33
           13471          863.2
           14911         773.54
2011-12-02 15769         3404.0
           13868         2685.0
           15856        2004.98

In [465]:
mod_df = df.loc[:]
month_map = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sep', 10:'Oct', 11:'Nov', 12:'Dec'}
mod_df['Month'] = pd.to_datetime(mod_df['Date']).dt.month.map(month_map)
rev_per_month = mod_df.pivot_table(values='Revenue', index='Month', aggfunc='sum').sort_values('Revenue', ascending=False)
rev_per_month.to_csv('../reports/tables/rev_per_month.csv')
rev_per_month

,Revenue
Month,
Nov,1156205.61
Dec,1087613.17
Oct,1035642.45
Sep,950690.202
May,677355.15
Jun,660046.05
Aug,644051.04
Jul,598962.901
Mar,594081.76


In [466]:
mod_df = df.loc[:]
pareto_df = mod_df.groupby('CustomerID')['Revenue'].sum().sort_values(ascending=False).reset_index()

pareto_df['cum_customers'] = (pareto_df.index + 1) / len(pareto_df)
pareto_df['cum_revenue'] = pareto_df['Revenue'].cumsum() / pareto_df['Revenue'].sum()

cutoff = pareto_df[pareto_df['cum_revenue'] >= 0.8].iloc[0]
cutoff

CustomerID        15611.0
Revenue           1591.45
cum_customers    0.260489
cum_revenue      0.800099
Name: 1129, dtype: Float64

## 9. Summary of Insights (write-up)
- **Time span analyzed:** <br> 2010-12-01 to 2011-12-09
- **Unique invoices/customers/products/countries:** <br> 18,532 invoices / 4,338 customers / 3,665 products / 37 countries
- **Top-selling products (by quantity & revenue):** <br>
    - By quantity: PAPER CRAFT , LITTLE BIRDIE / MEDIUM CERAMIC TOP STORAGE JAR / WORLD WAR 2 GLIDERS ASSTD DESIGNS
    - By revenue: PAPER CRAFT , LITTLE BIRDIE / REGENCY CAKESTAND 3 TIER / WHITE HANGING HEART T-LIGHT HOLDER
- **Countries with the most transactions / revenue:** <br>
    - Transactions: United Kingdom, Germany, France
    - Revenue: United Kingdom, Netherlands, EIRE
- **Pareto Test for customers:** <br> Approximately 26% of customers account for 80% of the total revenue, confirming a Pareto-like distribution in customer value.
- **Limitations/Notes:** <br>
    - The dataset only covers transactions between December 2010 and December 2011, so all insights are limited to this period and may not reflect longer-term customer behavior.
    - Cancelled invoices were not considered in this analysis, the results are focused only in completed transactions.
    - Some country values were “Unspecified” or recorded as “European Community”, which do not represent individual countries. These records were excluded from the country analysis to ensure accurate results.